In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 86 kB 2.9 MB/s 
     |████████████████████████████████| 6.6 MB 39.5 MB/s 
     |████████████████████████████████| 596 kB 48.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 5.2 MB/s 


In [3]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

from transformers import pipeline
from tqdm import tqdm
from torchmetrics import F1Score
from transformers import BertTokenizer, BertForSequenceClassification

In [4]:
model = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny-toxicity')
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.0M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29564, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, element

In [5]:
sentiment = pipeline("text-classification", model='cointegrated/rubert-tiny-toxicity')
sentiment(["Этот ресторан отличный", 'А тот ресторан - полная жопа'])

Downloading:   0%|          | 0.00/377 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': 'non-toxic', 'score': 0.9993882179260254},
 {'label': 'insult', 'score': 0.9857767224311829}]

In [6]:
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny-toxicity')

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[    2,  3086, 10885, 22723,   871, 24302,  3464, 10880,     3,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])


In [7]:
example_text = tokenizer.decode(bert_input.input_ids[0])
print(example_text)

[CLS] Пример текста для токенизации [SEP] [PAD]


In [8]:
from google.colab import drive
drive.mount('/content/drive')

!unzip /content/drive/MyDrive/05_pytorch/lesson_09/data.zip

df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

Mounted at /content/drive
Archive:  /content/drive/MyDrive/05_pytorch/lesson_09/data.zip
  inflating: train.csv               
  inflating: val.csv                 


((181467, 3), (22683, 3))

In [9]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [10]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [11]:
idx = 35
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
sentiment(df_train.iloc[idx]['text'])

@KravchenkoNasty да #echelonlovesfathers Ну у меня одна подруга слушает рок, ахах, я её прям люблю :D #echelonlovesfathers
label is 1


[{'label': 'non-toxic', 'score': 0.9904237389564514}]

In [12]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())

In [13]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny-toxicity')
               #для каждого text возвращает батч с полями:
               #'inputs_ids' -- тензор размера (B,1,max_len) из id токенов
               #'token_type_ids' -- тензор размера (B,1,max_len) из id типов токенов
               #'attention_mask' -- тензор размера (B,1,max_len) из индексов, указывающих, на какие токеты модель должна обратить внима
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [14]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [15]:
for txt, lbl in train_loader:
    print(txt.keys()) #словарь с ключами 'input_ids', 'token_type_ids', 'attention_mask'
    print(txt['input_ids'].shape) #тензор размера (B,1,max_len) из id токенов
    print(txt['attention_mask'].shape) #тензор размера (B,1,max_len) из индексов, указывающих, на какие токеты модель должна обратить внимание
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])
torch.Size([64, 1, 10])


In [93]:
class BertForSequenceClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.pretrained_model = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny-toxicity')
        self.dropout = nn.Dropout(dropout)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        pooled_output = self.pretrained_model(input_ids=x, attention_mask=mask, return_dict=False)[0]  #(B, 2)
        dropout_output = self.dropout(pooled_output)
        out = self.sigm(dropout_output)
        return out

In [94]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [96]:
model = BertForSequenceClassifier().to(device)
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.pretrained_model.classifier.parameters()]))

BertForSequenceClassifier(
  (pretrained_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(29564, 312, padding_idx=0)
        (position_embeddings): Embedding(512, 312)
        (token_type_embeddings): Embedding(2, 312)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=312, out_features=312, bias=True)
                (key): Linear(in_features=312, out_features=312, bias=True)
                (value): Linear(in_features=312, out_features=312, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=312, out

In [97]:
f1_val = F1Score().to(device)
model.eval()

total_acc_val = 0

for val_input, val_label in valid_loader:

    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze().to(device)
    output = model(input_id, mask)
    f1_val(output, val_label)
    acc = (output.argmax(dim=1) == val_label).sum().item()
    total_acc_val += acc
    
print(f'Val accuracy: {total_acc_val/len(valid_dataset):.3f}, F1 score: {f1_val.compute().item():.3f}')

Val accuracy: 0.471, F1 score: 0.471


In [98]:
#компиляция модели
criterion = nn.CrossEntropyLoss()

# оптимизатор для последнего слоя модели
optimizer = Adam(model.pretrained_model.parameters(), lr=0.0001)

In [99]:
f1_train = F1Score().to(device)
f1_valid = F1Score().to(device)

epochs = 5

for epoch_num in range(epochs):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)
        
        output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()

        f1_train(output, train_label)

        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

        f1_valid(output, val_label)

    # выведем метрики      
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Train f1: {f1_train.compute().item(): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f} \
        | Val f1: {f1_valid.compute().item(): .3f}')
    
    f1_train.reset()
    f1_valid.reset()

100%|██████████| 2836/2836 [01:01<00:00, 46.28it/s]


Epochs: 1 | Train Loss:  0.022         | Train Accuracy:  0.494         | Train f1:  0.494         | Val Loss:  0.018         | Val Accuracy:  0.494         | Val f1:  0.494


100%|██████████| 2836/2836 [01:00<00:00, 46.77it/s]


Epochs: 2 | Train Loss:  0.021         | Train Accuracy:  0.498         | Train f1:  0.498         | Val Loss:  0.018         | Val Accuracy:  0.510         | Val f1:  0.510


100%|██████████| 2836/2836 [01:01<00:00, 45.96it/s]


Epochs: 3 | Train Loss:  0.021         | Train Accuracy:  0.499         | Train f1:  0.499         | Val Loss:  0.018         | Val Accuracy:  0.496         | Val f1:  0.496


100%|██████████| 2836/2836 [01:00<00:00, 46.82it/s]


Epochs: 4 | Train Loss:  0.021         | Train Accuracy:  0.501         | Train f1:  0.501         | Val Loss:  0.018         | Val Accuracy:  0.563         | Val f1:  0.563


100%|██████████| 2836/2836 [01:00<00:00, 46.74it/s]


Epochs: 5 | Train Loss:  0.021         | Train Accuracy:  0.499         | Train f1:  0.499         | Val Loss:  0.018         | Val Accuracy:  0.555         | Val f1:  0.555


In [100]:
# считаю метрику дообученной модели на валидационном датасете

valid_f1 = F1Score().to(device)
model.eval()

total_acc_val = 0

for val_input, val_label in valid_loader:
  
    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze().to(device)
    output = model(input_id, mask)
    valid_f1(output, val_label)
    acc = (output.argmax(dim=1) == val_label).sum().item()
    total_acc_val += acc
    
print(f'Val accuracy: {total_acc_val/len(valid_dataset):.3f}, F1 score: {valid_f1.compute().item():.3f}')

Val accuracy: 0.555, F1 score: 0.555


дообученная модель показла метрику F1 score: 0.555, обычная модель имела метрику F1 score: 0.471